In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import UnivariateFeatureSelector
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [ ]:
selector = UnivariateFeatureSelector(
    featuresCol="temp_features",
    outputCol="features",
)
selector.setFeatureType("continuous").setLabelType("continuous").setSelectionThreshold(
    5
)

linear_model = LinearRegression()
linear_model.setRegParam(0.1)
pipeline = Pipeline(stages=[selector, linear_model])
paramGrid = (
    ParamGridBuilder()
    .addGrid(
        selector.selectionThreshold, [i for i in range(15, len(feature_columns) + 1)]
    )
    .addGrid(linear_model.regParam, [0.1])
    .addGrid(linear_model.solver, ["normal"])
    .build()
)
crossval = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=RegressionEvaluator(),
    numFolds=2,
)
cv_model = crossval.fit(df_assembler)